In [6]:
import time
from selenium import webdriver # importing webdriver 
from bs4 import BeautifulSoup as bs # using beautiful soup for web scraping
import re
import pandas as pd
from pandas import ExcelWriter
import datetime
import os

dt = datetime.datetime.now().strftime("%Y/%m/%d-%H:%M:%S")
filename = 'institute_details1'
writer = pd.ExcelWriter(filename+'.xlsx')
chrome_path = r'C:\Users\user\chromedriver.exe' # selecting web driver for chrome
browser = webdriver.Chrome(chrome_path)
time.sleep(1)

link="https://www.exambazaar.com/"
browser.get(link)

source_data = browser.page_source

bs_data = bs(source_data, 'lxml')

soup = bs_data.find_all('div',{'class':'p0Button width100 layout-align-center-center layout-row'})
lst_dict = []
category_link = []
try:
    for i in range(len(soup)):

        l = soup[i].get('href')
        CATEGORY_URL = link+l
        CATEGORY_URL = CATEGORY_URL.replace('gurgaon','delhi')
        a = CATEGORY_URL.split("/")[4]

        print("first :" + CATEGORY_URL)
        brow = webdriver.Chrome(chrome_path)
        time.sleep(2)
        brow.get(CATEGORY_URL)

        lenOfPage = brow.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match=False
        while(match==False):
            lastCount = lenOfPage
            time.sleep(20)
            lenOfPage = brow.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

        link_data = brow.page_source        
        bs_data = bs(link_data, 'lxml')
        h = bs_data.find_all('div',{'class': 'width100 brandBottomBorder2 layout-align-start-start layout-column'})
#         h = bs_data.find_all('div',{'class': 'width100 layout-xs-column layout-sm-column layout-md-column layout-gt-md-row layout-align-start-start'})
        brow.close()
        print("second :" + len(h))
        try:
            for j in range(len(h)):
                ins = h[j].find('a').get('href')
                print(ins)
                
                INSTITUTE_URL = link+ins
                brows = webdriver.Chrome(chrome_path)
                time.sleep(2)
                brows.get(link+ins)
                time.sleep(12)
                so=bs(brows.page_source, 'lxml')
                h1=so.find_all('div',{'class': 'width100 layout-align-start-start layout-column flex-xs-100 flex-sm-100 flex-md-50 flex-gt-md-70'})
                print(j)
                time.sleep(1)
                institute_name = brows.find_elements_by_xpath('//*[@id="exambazaar"]/div[1]/div[2]/div/div/div/div/div[3]/div[1]/h1')
                ins_name = institute_name[0].text.split("\n")[0]
                INSTITUTE_NAME = ins_name
                time.sleep(2)
        #         brows.close()
                PHONE = ""
                MOBILE =""
                TOTAL_CENTRE = ""
                ADDRESS = ""
                INSTITUTE_WEBSITE = ""
                EMAIL = ""

                if len(h1) != 0 :
                    web = h1[0].find('a',{'title': ins_name+ " " +'Website'})
                    if web is not None:
                        INSTITUTE_WEBSITE = web.get('href')
                    else:
                        INSTITUTE_WEBSITE = "na"
                else:
                    INSTITUTE_WEBSITE = "na"
                time.sleep(1)

                if len(h1) != 0 :
                    mail = h1[0].find_all('div',{'class': 'wrap toLowercase ng-binding flex'})
                    if len(mail) != 0:
                        if(re.search(r'[\w.-]+@[\w.-]+.\w+', mail[0].text)):
                            EMAIL = mail[0].text
                    else:
                        EMAIL = "na"
                else:
                    EMAIL = "na"

                time.sleep(1)

                if len(h1) != 0 :
                    ac = h1[0].find_all('div',{'class': 'wrap ng-binding flex'})
                    for i in range(len(ac)):
                        if(re.search("\d{3}-\d{7}",ac[i].text.strip()) or re.search("^(\d{11})$",ac[i].text.strip()) or re.search("\d{3,4}[- ]\d{3,4}[- ]\d{3,4}",ac[i].text.strip())):
                            if len(ac[i].text.strip()) != 0:
                                PHONE = ac[i].text.strip()
                            else:
                                PHONE = "na"
                        elif(re.search("^(\d{10},)*\d{10}$",ac[i].text.strip()) or re.search("((91|\+91|\(\+91\))?\s?\d{10})",ac[i].text.strip())):
                            if len(ac[i].text.strip()) != 0:
                                MOBILE = ac[i].text.strip()
                            else:
                                MOBILE = "na"
                        elif((re.search("[cC]entre", ac[i].text.strip()) or re.search("[iI]ndia", ac[i].text.strip())) and len(ac[i].text.strip())<=22):
                            if len(ac[i].text.strip()) != 0:
                                TOTAL_CENTRE = ac[i].text.strip()
                            else:
                                TOTAL_CENTRE = "na"
                        else:
                            if len(ac[i].text.strip()) != 0:
                                ADDRESS = ac[i].text.strip()
                            else:
                                ADDRESS = "na,"

                
                lst_dict.append({'INSTITUTE_CATEGORY_URL':CATEGORY_URL, 'INSTITUTE_URL':INSTITUTE_URL, 'INSTITUTE_WEBSITE': INSTITUTE_WEBSITE,'INSTITUTE_NAME':INSTITUTE_NAME,'INSTITUTE_EMAIL':EMAIL,'INSTITUTE_PHONE':PHONE,'INSTITUTE_MOBILE':MOBILE,'TOTAL_CENTRE':TOTAL_CENTRE,'INSTITUTE_ADDRESS':ADDRESS.replace(",", "|")})
#                 print(lst_dict)
                time.sleep(10)
                brows.close()
                
        except :
            pass

        df = pd.DataFrame(lst_dict)
#         print(df)
        df.to_excel(writer,str(a),columns=['INSTITUTE_CATEGORY_URL','INSTITUTE_URL','INSTITUTE_WEBSITE','INSTITUTE_NAME','INSTITUTE_EMAIL','INSTITUTE_PHONE','INSTITUTE_MOBILE','TOTAL_CENTRE','INSTITUTE_ADDRESS'])
        lst_dict = []
    writer.save()    
except :
    
    pass
    writer.save()


first :https://www.exambazaar.com//iit-jee-advanced-coaching/delhi
